In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.cm import Greens_r
from getters import LDLFI
pd.set_option('display.expand_frame_repr', False)

# Collecting and plotting

## Collecting

Our goal within this notebook is to first collect the information from [this table]("https://lfi.ch/resultate/resultate-en.php?befNr=96&fldSearch=&invNr=450&p=search&regionNr=366&zigrNr=21&prodNr=70&prodItNr=869392") about the volume within the economic regions, the accessible forests without shrub forest, and during the latest published NFI period classified by conifers and broadleaves.

To fetch the data, we will use the keys that are used in the underlying relational database. To get these keys, we wrote some small functions to help in this.

First, we are interested in the number of our topic:

In [ ]:
LDLFI.getTopicNumbers()